Install ffmpeg and libavcodec-dev (plus related stuff) from
https://launchpad.net/~mc3man/+archive/ubuntu/trusty-media

Install OpenCV using the instructions from 
http://www.pyimagesearch.com/2015/06/22/install-opencv-3-0-and-python-2-7-on-ubuntu/

In my case I had a build error related to missing CUDA libs, so the build command I used was

    cmake -D CMAKE_BUILD_TYPE=RELEASE \
            -D CMAKE_INSTALL_PREFIX=/usr/local \
            -D INSTALL_C_EXAMPLES=OFF \
            -D INSTALL_PYTHON_EXAMPLES=ON \
            -D "OPENCV_EXTRA_MODULES_PATH=../../opencv_contrib/modules" \
            -D BUILD_EXAMPLES=ON \
            -D WITH_CUDA=OFF ..



In [1]:
!sudo pip install pytube

Cleaning up...


In [2]:
# Reset GPU
#!sudo rmmod nvidia_uvm
#!sudo rmmod nvidia
#!sudo insmod /lib/modules/3.13.0-77-generic/kernel/drivers/video/nvidia.ko
#!sudo insmod /lib/modules/3.13.0-77-generic/kernel/drivers/video/nvidia-uvm.ko
#!echo "OK"

In [3]:
!mkdir videos
!mkdir imageseries

mkdir: cannot create directory 'videos': File exists
mkdir: cannot create directory 'imageseries': File exists


In [4]:
!ls videos

Capoeira_Girl.mp4   Chimpanzees1.mp4		     Solar-System.mp4
Capoeira_Movie.mp4  Chimpanzees1_reconstructed2.mp4
Chimpanzees.mp4     Chimpanzees1_reconstructed3.mp4


In [5]:
videos = {
    'Solar-System' : 'https://www.youtube.com/watch?v=jvhM4tANYDg',
    'Chimpanzees' : 'https://www.youtube.com/watch?v=yim23IFJOyQ',
    'Capoeira_Girl' : 'https://www.youtube.com/watch?v=vwG7JwXtUSI',
    'Capoeira_Movie' : 'https://www.youtube.com/watch?v=6-DYrZXj3W4'
}
download = False

In [6]:
current_movie = 'Chimpanzees.mp4'

In [7]:
if (download):

    from pytube import YouTube

    # not necessary, just for demo purposes.
    from pprint import pprint
    for filename in videos.keys():
        url = videos[filename]
        yt = YouTube(url)
        print "Downloading %s from %s" % (filename, url)
        yt.set_filename(filename)
        video = yt.get('mp4', '360p')
        video.download('./videos/')


In [8]:
# Now let's try to read this using our fresh opencv build..

In [9]:
import cv2
import h5py


In [10]:
from IPython.display import HTML, FileLink, display, clear_output
from base64 import b64encode
def link_video(path):
    src = FileLink(path)
    video_tag = '<video controls alt="test" src="%s">' % (path)
    
    display(HTML(data=video_tag))


In [11]:
vc = cv2.VideoCapture('videos/%s' % current_movie)
frames = 0
shape = None
dtype = None
while (True):
    ret, img = vc.read()
    # img should be an numpy ndarray of shape (width, heigth, channels)
    if (not ret):
        break
    if (shape is None):
        shape = img.shape
    if (dtype is None):
        dtype = img.dtype
    if (frames>=15000):
        break
    frames+=1
    
    
print "Could read %d frames of shape %r and dtype %r" % (frames, shape, dtype)

Could read 1490 frames of shape (360, 640, 3) and dtype dtype('uint8')


In [12]:
#import h5py

#h5f = h5py.File('videos/Chimpanzees1_7.hdf5',driver='core', backing_store=True)
#h5f = h5py.File('/tmp/Chimpanzees1_7.hdf5')
#h5g = h5f.create_group('input')

In [13]:
import numpy as np

In [14]:
rshape = [ shape[2],shape[0], shape[1] ]

In [15]:
ishape = tuple([ frames ] + list(rshape))
ichunkshape = tuple([ 1 ] + list(rshape))
imaxshape = tuple([ None ] + list(rshape))

In [16]:
import numpy as np
from numpy.random import permutation


In [17]:
#inframes = h5g.create_dataset('raw_hsl_frames', 
#                             shape=ishape, dtype=np.float32, 
#                             chunks=ichunkshape, maxshape=imaxshape,
#                             fletcher32=True, fillvalue=0)
#
inframes = np.zeros(shape=ishape, dtype=np.float32)

In [18]:
nframes = frames

In [19]:
frames

1490

In [20]:
inframes.shape


(1490, 3, 360, 640)

In [21]:
permuted_indices = permutation(range(frames))

permuted_indices

array([ 759,  257, 1234, ..., 1482, 1313,  993])

In [22]:
nframes

1490

In [23]:
vc = cv2.VideoCapture('videos/%s' % current_movie)
frames = 0
shape = None
dtype = None
for f in range(nframes):
    ret, img = vc.read()
    if (not ret):
        break
    hsvimg = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    
    hsvimgf = hsvimg.astype(np.float32) / 255.0
    hsvimgf = hsvimgf.swapaxes(2,0).swapaxes(1,2)
    # img should be an numpy ndarray of shape (width, heigth, channels)
    inframes[permuted_indices[frames],:,:,:] = hsvimgf
    
    #hsvimg2 = (inframes[permuted_indices[frames],:,:,:] * 255.0).astype(np.uint8)
    
    #print np.sum(hsvimg-hsvimg2)
    #bgrimg = cv2.cvtColor(hsvimg2, cv2.COLOR_HSV2BGR)
    #print np.sum(img-bgrimg)
    
    #cv2.imwrite("videos/recoded1_%05d.png" % (frames), bgrimg)
    
    frames+=1
    if (frames % 300==0):
        print "Frame #%d" % (frames)
print "Read %d frames" % (frames)


Frame #300
Frame #600
Frame #900
Frame #1200
Read 1490 frames


In [24]:
#inframesmem = np.array(inframes)

In [25]:
#!rm videos/Chimpanzees1_recoded.mp4
#!ffmpeg -i imageseries/Chimpanzees1_%05d.png -c:v libx264 -b:v 400k -vf format=yuv420p -maxrate:v 500k -bufsize 1000k  -r 10 -preset medium videos/Chimpanzees1_recoded.mp4

In [26]:
#link_video('videos/Chimpanzees1_recoded.mp4')

In [27]:
import os
import h5py
from glob import glob
import numpy as np


In [28]:
from numpy.lib.stride_tricks import as_strided

def tile_array(a, b1, b2):
    r, c = a.shape
    rs, cs = a.strides
    x = as_strided(a, (r, b1, c, b2), (rs, 0, cs, 0))
    return x.reshape(r*b1, c*b2)

def tile_batch_mask(shape, p, tile_x, tile_y):
    target = np.ones(shape=shape, dtype=np.float32)
    assert(shape[1]==1)
    for i in range(target.shape[0]):
        
        rnd_shape = (shape[2] / tile_x, shape[3] / tile_y)
        rnd = (np.random.random(rnd_shape)<=p).astype(np.float32)
        tiled = tile_array(rnd, tile_x, tile_y)
        target[i,0,:,:] = tiled
    return target

## MXNet ##



In [29]:
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.DEBUG, datefmt='%I:%M:%S')
logging.debug("debug")
logging.info("info")

import os
# Comment this out, once it all works
os.environ['MXNET_ENGINE_TYPE']='NaiveEngine'


06:30:00 DEBUG:debug
06:30:00 INFO:info


In [30]:
logger = logging.getLogger()


In [31]:
import mxnet as mx
import mxnet.io as mxio


In [ ]:

def create_network(dilate_layers=4, squeeze_factor = 0.5, mult_factor=4, squeeze_constant=16, 
                   conv_factor=24, conv_constant=32, intermediate_dropout=0.05, 
                   final_dropout=0.2, final_num_features=1024):

    data = mx.symbol.Variable('data')
    y = mx.symbol.Variable('y')

    net = data
    dil = 1
    for d in range(1,dilate_layers+1):
        
        #net = mx.symbol.Convolution(net, kernel=(3,3), num_filter=int(d*squeeze_factor*mult_factor)+squeeze_constant, name='squeeze_pre_%d' % (d), workspace=200)
        
        
        
        #netA = mx.symbol.Convolution(net, kernel=(3,3), num_filter=d*conv_factor/2+conv_constant, 
        #                            dilate=(dil,dil), name='convA%d' % (d), workspace=200)
        
        #netB = mx.symbol.Convolution(net, kernel=(1,1), num_filter=d*conv_factor/2+conv_constant, 
        #                            dilate=(dil,dil), name='convB%d' % (d), workspace=200)

        # We need to crop net1, since it can be larger than net2 
        #netB = mx.symbol.Crop(netB, netA, num_args=2, center_crop=True)  

        #net = mx.symbol.Concat(netA,netB)

        #net = mx.symbol.LeakyReLU(net, act_type='prelu')
        #print net.infer_shape(data=(10,4,64,64))
        #if (intermediate_dropout>0.0):
            #net = mx.symbol.DropoutAllChannels(net, p=intermediate_dropout)
        
        #net = mx.symbol.Convolution(net, kernel=(1,1), num_filter=(d+1)*mult_factor+squeeze_constant, name='squeeze_dummy_%d' % (d), workspace=200)
        #print "PX1: %d" % ( int(d*squeeze_factor*mult_factor)+squeeze_constant)
        #print "PX1: %d" % ( (d+1)*mult_factor+squeeze_constant)
        #print net.infer_shape(data=(10,4,64,64))
        #net = mx.symbol.Convolution(net, kernel=(1,1),  num_filter=(d+1)*mult_factor+squeeze_constant, name='squeeze_pre_%d' % (d), workspace=200)
        
        net = mx.symbol.Convolution(net, kernel=(3,3), dilate=(2,2), num_filter=(d+1)*mult_factor+squeeze_constant, name='squeeze_post_%d' % (d), workspace=200)
        net = mx.symbol.LeakyReLU(net, act_type='prelu')
        
        dil *= 2

    net = mx.symbol.Convolution(net, kernel=(3,3), num_filter=final_num_features, name='squeeze_final', workspace=200)
    if (final_dropout>0.0):
        net = mx.symbol.Dropout(net, p=final_dropout)
    net = mx.symbol.Activation(net, act_type='relu')
    
    net = mx.symbol.Convolution(net, kernel=(3,3), num_filter=3, name='decoder',  workspace=200)
    net = mx.symbol.Activation(net, name='final_activation', act_type='relu')
    #crop_y = mx.symbol.Crop(y, net, num_args=2, center_crop=True)
    out = mx.symbol.MAERegressionOutput(data=net, label=y, name='output_')
    return out, net
    
def get_output_resolution(net, input_shape=(10, 3, 640, 480)):
    mshape = list(input_shape)
    mshape[1] = 1
    return net.infer_shape(data=input_shape, mask=mshape)[-2][0][2:]

def get_middle_slice(net, X):
    res = get_output_resolution(net, X.shape)
    dx = (X.shape[2]-res[0]) // 2
    dy = (X.shape[3]-res[1]) // 2
    return X[:,:,dx:X.shape[2]-dx,dy:X.shape[3]-dy]




In [33]:
from myio import NDArrayGeneratorIter

In [109]:


def sample_hyperparams(hyperparam_options):
    hyperparams = {}
    for key in hyperparam_options.keys():
        hyperparams[key] = hyperparam_options[key][randint(0, len(hyperparam_options[key]))]
    return hyperparams


def eval_hyperparams(inframes, hyperparams, num_epoch=10, windowsize=96, batch_size=32, 
                     keep_prob=0.9, drop_pixel_x=2, drop_pixel_y=2):

    hparams = dict(hyperparams)
    del hparams['weight_decay']
    chunk_size = batch_size*4
    print "Creating Model with params %r" % (hyperparams)
    out, net = create_network(**hparams)
    eval_metric = mx.metric.MAE()
    
    allgpus = [ mx.gpu(i) for i in range(2)]
    allcpus = [ mx.cpu(i) for i in range(12)]
    optimizer = myopt.MyAdaGrad(learning_rate=0.01, wd=hyperparams['weight_decay'])
    model = mxmodel.FeedForward(ctx=allgpus, symbol=out, 
                                num_epoch=num_epoch, optimizer=optimizer)
    
    def training_generator():
        while True:
            window_off_x =  np.random.randint(0, inframes.shape[2]-windowsize)
            window_off_y = np.random.randint(0, inframes.shape[3]-windowsize)
            start = np.random.randint(0, inframes.shape[0]-400-chunk_size)
            train_X = inframes[start:start+chunk_size,:,window_off_x:window_off_x+windowsize,window_off_y:window_off_y+windowsize]
            train_y = get_middle_slice(net, train_X)
            mask_shape = list(train_X.shape)
            mask_shape[1] = 1
            mask = tile_batch_mask(mask_shape, keep_prob, drop_pixel_x, drop_pixel_y)

            #train_X = np.concatenate((mask, train_X*mask), axis=1)
            yield {'data' : train_X}, { 'y' : train_y}

    def test_generator():
        while True:
            window_off_x =  np.random.randint(0, inframes.shape[2]-windowsize)
            window_off_y = np.random.randint(0, inframes.shape[3]-windowsize)
            test_X = inframes[inframes.shape[0]-400:,:,window_off_x:window_off_x+windowsize,window_off_y:window_off_y+windowsize]
            test_y = get_middle_slice(net, test_X)
            test_mask_shape = list(test_X.shape)
            test_mask_shape[1] = 1
            mask = tile_batch_mask(test_mask_shape, keep_prob, drop_pixel_x, drop_pixel_y)

            #test_X = np.concatenate((test_mask, test_X*test_mask), axis=1)
            yield {'data' : test_X}, { 'y' : test_y}
            
   
        #test_X = np.concatenate((test_mask, test_X*test_mask), axis=1)
    train_iter = NDArrayGeneratorIter(training_generator(), 2, shuffle=True, batch_size=batch_size, last_batch_handle='discard')
    test_iter = NDArrayGeneratorIter(training_generator(), 1, shuffle=True, batch_size=batch_size, last_batch_handle='discard')
    test_iter.reset()
    model.fit(train_iter,
              eval_data=test_iter,
              eval_metric=eval_metric,
              batch_end_callback=mx.callback.Speedometer(batch_size, 4))
        
    return model, hyperparams, eval_metric.get()


In [39]:
import traceback
from IPython.display import clear_output,display
from numpy.random import randint
import mymodel as mxmodel
import myopt as myopt


In [103]:
results = []

In [37]:
logger.level = logging.INFO

In [113]:

hyperparam_options = {
    'dilate_layers' : [1],
    'squeeze_factor' : [ 0.5, 1.0 ],
    'mult_factor' : [ 2],
    'squeeze_constant' : [ 8 ],
    'conv_factor' : [ 16],
    'conv_constant' : [ 8],
    'intermediate_dropout' : [ 0.0],
    'final_dropout' : [0.1],
    'final_num_features' : [ 160],
    'weight_decay' : [0.0]
}


for i in range(30):
    hyperparams = sample_hyperparams(hyperparam_options)
    display(results)
    print "Starting with %r" % (hyperparams)
    try:
        res = eval_hyperparams(inframes, num_epoch=20, keep_prob=1.0, hyperparams=hyperparams, windowsize=96)
        results.append(res)
        model,hparams, metric = res
        display("Result %d: Validation Accuracy: %r, Params: %r " % (i, metric, hparams))
    except Exception as e:
        print "Exception for hyperparams %r" % (hyperparams)
        results.append((None, hyperparams, None))
        traceback.print_exc(e)
    

[(FeedForward(allow_extra_params=False,
        arg_params={'squeeze_final_bias': <mxnet.ndarray.NDArray object at 0x2b5dbdef0ad0>, 'decoder_weight': <mxnet.ndarray.NDArray object at 0x2b5dbdef0b10>, 'decoder_bias': <mxnet.ndarray.NDArray object at 0x2b5dbdef0a10>, 'squeeze_post_1_bias': <mxnet.ndarray.NDArray object at 0x2b5dbdef0f10>, 'leakyrel...bject at 0x2b5dbdef0190>, 'squeeze_post_1_weight': <mxnet.ndarray.NDArray object at 0x2b5dbdef0690>},
        aux_params={}, begin_epoch=0, ctx=[gpu(0)], epoch_size=None,
        initializer=<mxnet.initializer.Uniform object at 0x2b5dbdc31dd0>,
        num_epoch=20, numpy_batch_size=128,
        optimizer=<myopt.MyAdaGrad object at 0x2b5dc4998410>,
        symbol=<mxnet.symbol.Symbol object at 0x2b5dc4998110>),
  {'conv_constant': 8,
   'conv_factor': 16,
   'dilate_layers': 1,
   'final_dropout': 0.1,
   'final_num_features': 160,
   'intermediate_dropout': 0.0,
   'mult_factor': 2,
   'squeeze_constant': 8,
   'squeeze_factor': 0.5,
   'we

07:38:40 INFO:Auto-select kvstore type = local_update_cpu
07:38:40 INFO:Start training with [gpu(0), gpu(1)]


Starting with {'squeeze_factor': 1.0, 'final_dropout': 0.1, 'intermediate_dropout': 0.0, 'conv_constant': 8, 'squeeze_constant': 8, 'mult_factor': 2, 'final_num_features': 160, 'conv_factor': 16, 'dilate_layers': 1, 'weight_decay': 0.0}
Creating Model with params {'squeeze_factor': 1.0, 'final_dropout': 0.1, 'intermediate_dropout': 0.0, 'conv_constant': 8, 'squeeze_constant': 8, 'mult_factor': 2, 'final_num_features': 160, 'conv_factor': 16, 'dilate_layers': 1, 'weight_decay': 0.0}


07:38:41 INFO:Epoch[0] Batch [4]	Speed: 309.47 samples/sec	Train-mae=0.368319
07:38:41 INFO:Epoch[0] Batch [8]	Speed: 222.79 samples/sec	Train-mae=0.316883
07:38:42 INFO:Epoch[0] Batch [12]	Speed: 226.76 samples/sec	Train-mae=0.201090
07:38:42 INFO:Epoch[0] Resetting Data Iterator
07:38:42 INFO:Epoch[0] Time cost=1.697
07:38:42 INFO:Epoch[0] Validation-mae=0.190086
07:38:43 INFO:Epoch[1] Batch [4]	Speed: 311.42 samples/sec	Train-mae=0.164065
07:38:43 INFO:Epoch[1] Batch [8]	Speed: 227.46 samples/sec	Train-mae=0.161994
07:38:44 INFO:Epoch[1] Batch [12]	Speed: 226.97 samples/sec	Train-mae=0.135071
07:38:44 INFO:Epoch[1] Resetting Data Iterator
07:38:44 INFO:Epoch[1] Time cost=1.680
07:38:44 INFO:Epoch[1] Validation-mae=0.146283
07:38:45 INFO:Epoch[2] Batch [4]	Speed: 311.99 samples/sec	Train-mae=0.131707
07:38:45 INFO:Epoch[2] Batch [8]	Speed: 227.48 samples/sec	Train-mae=0.122608
07:38:46 INFO:Epoch[2] Batch [12]	Speed: 227.01 samples/sec	Train-mae=0.121574
07:38:46 INFO:Epoch[2] Resett

"Result 0: Validation Accuracy: ('mae', 0.034165979479439557), Params: {'squeeze_factor': 1.0, 'final_dropout': 0.1, 'intermediate_dropout': 0.0, 'conv_constant': 8, 'squeeze_constant': 8, 'mult_factor': 2, 'final_num_features': 160, 'conv_factor': 16, 'dilate_layers': 1, 'weight_decay': 0.0} "

...

[(FeedForward(allow_extra_params=False,
        arg_params={'squeeze_final_bias': <mxnet.ndarray.NDArray object at 0x2b5dbdef0ad0>, 'decoder_weight': <mxnet.ndarray.NDArray object at 0x2b5dbdef0b10>, 'decoder_bias': <mxnet.ndarray.NDArray object at 0x2b5dbdef0a10>, 'squeeze_post_1_bias': <mxnet.ndarray.NDArray object at 0x2b5dbdef0f10>, 'leakyrel...bject at 0x2b5dbdef0190>, 'squeeze_post_1_weight': <mxnet.ndarray.NDArray object at 0x2b5dbdef0690>},
        aux_params={}, begin_epoch=0, ctx=[gpu(0)], epoch_size=None,
        initializer=<mxnet.initializer.Uniform object at 0x2b5dbdc31dd0>,
        num_epoch=20, numpy_batch_size=128,
        optimizer=<myopt.MyAdaGrad object at 0x2b5dc4998410>,
        symbol=<mxnet.symbol.Symbol object at 0x2b5dc4998110>),
  {'conv_constant': 8,
   'conv_factor': 16,
   'dilate_layers': 1,
   'final_dropout': 0.1,
   'final_num_features': 160,
   'intermediate_dropout': 0.0,
   'mult_factor': 2,
   'squeeze_constant': 8,
   'squeeze_factor': 0.5,
   'we

07:40:01 INFO:Auto-select kvstore type = local_update_cpu
07:40:01 INFO:Start training with [gpu(0), gpu(1)]


Starting with {'squeeze_factor': 0.5, 'final_dropout': 0.1, 'intermediate_dropout': 0.0, 'conv_constant': 8, 'squeeze_constant': 8, 'mult_factor': 2, 'final_num_features': 160, 'conv_factor': 16, 'dilate_layers': 1, 'weight_decay': 0.0}
Creating Model with params {'squeeze_factor': 0.5, 'final_dropout': 0.1, 'intermediate_dropout': 0.0, 'conv_constant': 8, 'squeeze_constant': 8, 'mult_factor': 2, 'final_num_features': 160, 'conv_factor': 16, 'dilate_layers': 1, 'weight_decay': 0.0}


07:40:01 INFO:Epoch[0] Batch [4]	Speed: 309.89 samples/sec	Train-mae=0.456967
07:40:02 INFO:Epoch[0] Batch [8]	Speed: 226.00 samples/sec	Train-mae=0.294392
07:40:02 INFO:Epoch[0] Batch [12]	Speed: 227.68 samples/sec	Train-mae=0.232851
07:40:02 INFO:Epoch[0] Resetting Data Iterator
07:40:02 INFO:Epoch[0] Time cost=1.687
07:40:03 INFO:Epoch[0] Validation-mae=0.245401
07:40:03 INFO:Epoch[1] Batch [4]	Speed: 309.52 samples/sec	Train-mae=0.223417
07:40:04 INFO:Epoch[1] Batch [8]	Speed: 227.18 samples/sec	Train-mae=0.208210
07:40:04 INFO:Epoch[1] Batch [12]	Speed: 227.62 samples/sec	Train-mae=0.215847
07:40:04 INFO:Epoch[1] Resetting Data Iterator
07:40:04 INFO:Epoch[1] Time cost=1.682
07:40:05 INFO:Epoch[1] Validation-mae=0.232048
07:40:05 INFO:Epoch[2] Batch [4]	Speed: 312.73 samples/sec	Train-mae=0.222416
07:40:06 INFO:Epoch[2] Batch [8]	Speed: 227.93 samples/sec	Train-mae=0.206530
07:40:06 INFO:Epoch[2] Batch [12]	Speed: 208.57 samples/sec	Train-mae=0.209587
07:40:06 INFO:Epoch[2] Resett

KeyboardInterrupt: 

In [ ]:
from collections import defaultdict

In [ ]:
def analyze_results(results):

    param_analysis = defaultdict(lambda: defaultdict(lambda: []))
    for model, hyperparams, eval_metric in results:
        eval_result = 1.0
        if (eval_metric is not None):
        
            eval_result = eval_metric[1]
            
        for param in hyperparams.keys():
            value = hyperparams[param]
            param_analysis[param][value].append(eval_result)
            param_analysis[param][value] = sorted(param_analysis[param][value])
    return param_analysis

param_analysis = analyze_results(results)
    
    
    

In [ ]:
dict(param_analysis)

In [ ]:
 def adef():
        return []
        
def cdef():
       return defaultdict(adef)
  
param_analysis = defaultdict(default_factory=cdef)

In [ ]:
model

In [ ]:
net.list_arguments()

In [ ]:
get_middle_slice(net, inframes)

In [ ]:
model, hyperparams = eval_hyperparams(inframes, num_epoch=5, hyperparam_options=hyperparam_options)

In [ ]:
%debug


In [ ]:
tile_batch_mask((1,1,16,16), 0.3, 8,8)

In [ ]:
model.num_epoch = 50
model.load('dilated-autoencoder2_4', 2)

In [ ]:
model.num_epoch = 50

In [ ]:

chunk_size = batch_size*32
keep_prob = 0.9
drop_pixel_x = 2
drop_pixel_y = 2
for i in range(69):
    window_off_x =  np.random.randint(0, inframes.shape[2]-windowsize)
    window_off_y = np.random.randint(0, inframes.shape[3]-windowsize)
    start = np.random.randint(0, 1300-chunk_size)
    train_X = inframes[start:start+chunk_size,:,window_off_x:window_off_x+windowsize,window_off_y:window_off_y+windowsize]
    train_y = get_middle_slice(train_X)
    mask_shape = list(train_X.shape)
    mask_shape[1] = 1
    mask = tile_batch_mask(mask_shape, keep_prob, drop_pixel_y, drop_pixel_y)
    
    train_X = np.concatenate((mask, train_X*mask), axis=1)
    
    test_X = inframes[1300:,:,window_off_x:window_off_x+windowsize,window_off_y:window_off_y+windowsize]
    test_y = get_middle_slice(test_X)
    test_mask_shape = list(test_X.shape)
    test_mask_shape[1] = 1
    if (i%2==0):
        test_mask = tile_batch_mask(test_mask_shape, keep_prob, drop_pixel_y, drop_pixel_y)
    else:
        test_mask = tile_batch_mask(test_mask_shape, keep_prob, drop_pixel_y, drop_pixel_y)
    
    
    test_X = np.concatenate((test_mask, test_X*test_mask), axis=1)
    train_iter = mx.io.NDArrayIter({'data' : train_X}, { 'y' : train_y }, shuffle=True, batch_size=batch_size)
    test_iter = mx.io.NDArrayIter({'data' : test_X}, { 'y' : test_y}, shuffle=True, batch_size=batch_size)
    print "New Windowed Batch %i" % (i)
    eval_metric = mx.metric.MAE()
    model.fit(train_iter,
          eval_data=test_iter,
          eval_metric=eval_metric,
          batch_end_callback=mx.callback.Speedometer(batch_size))
    if (i % 5==4):
        model.save('dilated-autoencoder2_%d' % (i))

## MC Dropout Training / Validation Loop


In [ ]:
out.bind()

In [ ]:
executor=None

In [ ]:
def train_mc_dropout(mlp, ctx, train_iter, validation_iter, epochs=1,
                     opt = mx.optimizer.Adam(wd=0.01), eval_metric=mx.metric.MAE(), init = mx.init.Uniform(scale=0.01), executor=None):
    # data iterators defines shapes of its output with provide_data and provide_label property.
    input_shapes = dict(train_iter.provide_data+train_iter.provide_label)

    if (executor is not None):
        exe = executor
    else:
        # We use simple_bind to let MXNet allocate memory for us.
        # You can also allocate memory youself and use bind to pass it to MXNet.
        exe = mlp.simple_bind(ctx=ctx, **input_shapes)

    # ===============Initialization=================
    # First we get handle to input arrays
    arg_arrays = dict(zip(mlp.list_arguments(), exe.arg_arrays))
    data = arg_arrays[train_iter.provide_data[0][0]]
    label = arg_arrays[train_iter.provide_label[0][0]]

    if (executor is None):

        # We initialize the weights with uniform distribution on (-0.01, 0.01).

        for name, arr in arg_arrays.items():
            if name not in input_shapes:
                init(name, arr)

    # We also need to create an optimizer for updating weights
    
    updater = mx.optimizer.get_updater(opt)

    # Finally we need a metric to print out training progress
    metric = eval_metric

    # Training loop begines
    for epoch in range(epochs):
        train_iter.reset()
        metric.reset()
        t = 0
        for batch in train_iter:
            # Copy data to executor input. Note the [:].
            data[:] = batch.data[0]
            label[:] = batch.label[0]

            # Forward
            exe.forward(is_train=True)

            # You perform operations on exe.outputs here if you need to.
            # For example, you can stack a CRF on top of a neural network.

            # Backward
            exe.backward()

            # Update
            for i, pair in enumerate(zip(exe.arg_arrays, exe.grad_arrays)):
                weight, grad = pair
                updater(i, grad, weight)
            metric.update(batch.label, exe.outputs)
            t += 1
            if t % 10 == 0:
                print 'epoch:', epoch, 'iter:', t, 'metric:', metric.get()  
    return exe



In [ ]:
m = results[0][2]

In [ ]:
m.sum_metric

In [ ]:
executor = None

In [ ]:
executor

In [ ]:
from collections import OrderedDict

import ctypes
import sys
import numpy as np
import logging

from mxnet.ndarray import NDArray
from mxnet.ndarray import array
from mxnet.io import DataIter, DataBatch

class RandomImageSlicingNDArrayIter(DataIter):
    """NDArrayIter object in mxnet. Taking NDArray or numpy array to get dataiter.
    Parameters
    ----------
    data: NDArray or numpy.ndarray, a list of them, or a dict of string to them.
        NDArrayIter supports single or multiple data and label.
    label: NDArray or numpy.ndarray, a list of them, or a dict of them.
        Same as data, but is not fed to the model during testing.
    batch_size: int
        Batch Size
    shuffle: bool
        Whether to shuffle the data
    last_batch_handle: 'pad', 'discard' or 'roll_over'
        How to handle the last batch
    Note
    ----
    This iterator will pad, discard or roll over the last batch if
    the size of data does not match batch_size. Roll over is intended
    for training and can cause problems if used for prediction.
    """
    def __init__(self, data, label=None, batch_size=1, shuffle=False, last_batch_handle='pad'):
        # pylint: disable=W0201

        super(RandomImageSlicingNDArrayIter, self).__init__()

        self.data = _init_data(data, allow_empty=False, default_name='data')
        self.label = _init_data(label, allow_empty=True, default_name='softmax_label')

        # shuffle data
        if shuffle:
            idx = np.arange(self.data[0][1].shape[0])
            np.random.shuffle(idx)
            self.data = [(k, v[idx]) for k, v in self.data]
            self.label = [(k, v[idx]) for k, v in self.label]

        self.data_list = [x[1] for x in self.data] + [x[1] for x in self.label]
        self.num_source = len(self.data_list)

        # batching
        if last_batch_handle == 'discard':
            new_n = self.data_list[0].shape[0] - self.data_list[0].shape[0] % batch_size
            data_dict = OrderedDict(self.data)
            label_dict = OrderedDict(self.label)
            for k, _ in self.data:
                data_dict[k] = data_dict[k][:new_n]
            for k, _ in self.label:
                label_dict[k] = label_dict[k][:new_n]
            self.data = data_dict.items()
            self.label = label_dict.items()
        self.num_data = self.data_list[0].shape[0]
        assert self.num_data >= batch_size, \
            "batch_size need to be smaller than data size."
        self.cursor = -batch_size
        self.batch_size = batch_size
        self.last_batch_handle = last_batch_handle

    @property
    def provide_data(self):
        """The name and shape of data provided by this iterator"""
        return [(k, tuple([self.batch_size] + list(v.shape[1:]))) for k, v in self.data]

    @property
    def provide_label(self):
        """The name and shape of label provided by this iterator"""
        return [(k, tuple([self.batch_size] + list(v.shape[1:]))) for k, v in self.label]


    def hard_reset(self):
        """Igore roll over data and set to start"""
        self.cursor = -self.batch_size

    def reset(self):
        if self.last_batch_handle == 'roll_over' and self.cursor > self.num_data:
            self.cursor = -self.batch_size + (self.cursor%self.num_data)%self.batch_size
        else:
            self.cursor = -self.batch_size

    def iter_next(self):
        self.cursor += self.batch_size
        if self.cursor < self.num_data:
            return True
        else:
            return False

    def next(self):
        if self.iter_next():
            return DataBatch(data=self.getdata(), label=self.getlabel(), \
                    pad=self.getpad(), index=None)
        else:
            raise StopIteration

    def _getdata(self, data_source):
        """Load data from underlying arrays, internal use only"""
        assert(self.cursor < self.num_data), "DataIter needs reset."
        if self.cursor + self.batch_size <= self.num_data:
            return [array(x[1][self.cursor:self.cursor+self.batch_size]) for x in data_source]
        else:
            pad = self.batch_size - self.num_data + self.cursor
            return [array(np.concatenate((x[1][self.cursor:], x[1][:pad]),
                                         axis=0)) for x in data_source]

    def getdata(self):
        return self._getdata(self.data)

    def getlabel(self):
        return self._getdata(self.label)

    def getpad(self):
        if self.last_batch_handle == 'pad' and \
           self.cursor + self.batch_size > self.num_data:
            return self.cursor + self.batch_size - self.num_data
        else:
            return 0


In [ ]:
chunk_size = batch_size*32
keep_prob = 0.9
drop_pixel_x = 2
drop_pixel_y = 2

for i in range(3):
    window_off_x =  np.random.randint(0, inframes.shape[2]-windowsize)
    window_off_y = np.random.randint(0, inframes.shape[3]-windowsize)
    start = np.random.randint(0, inframes.shape[0]-1000-chunk_size)
    train_X = inframes[start:start+chunk_size,:,window_off_x:window_off_x+windowsize,window_off_y:window_off_y+windowsize]
    train_y = get_middle_slice(train_X)
    mask_shape = list(train_X.shape)
    mask_shape[1] = 1
    mask = tile_batch_mask(mask_shape, keep_prob, drop_pixel_y, drop_pixel_y)
    
    train_X = np.concatenate((mask, train_X*mask), axis=1)
    
    test_X = inframes[inframes.shape[0]-1000:inframes.shape[0]-800,:,window_off_x:window_off_x+windowsize,window_off_y:window_off_y+windowsize]
    test_y = get_middle_slice(test_X)
    test_mask_shape = list(test_X.shape)
    test_mask_shape[1] = 1
    if (i%2==0):
        test_mask = tile_batch_mask(test_mask_shape, keep_prob, drop_pixel_y, drop_pixel_y)
    else:
        test_mask = tile_batch_mask(test_mask_shape, keep_prob, drop_pixel_y, drop_pixel_y)
    
    
    test_X = np.concatenate((test_mask, test_X*test_mask), axis=1)
    train_iter = mx.io.NDArrayIter({'data' : train_X}, { 'y' : train_y }, shuffle=True, batch_size=batch_size)
    test_iter = mx.io.NDArrayIter({'data' : test_X}, { 'y' : test_y}, shuffle=True, batch_size=batch_size)
    print "New Windowed Batch %i" % (i)
    executor = train_mc_dropout(out, ctx=mx.gpu(0), train_iter=train_iter, validation_iter=test_iter, executor=executor)
    
    

In [ ]:
size = 0
for k in model.arg_params:
    size += np.prod(model.arg_params[k].shape)
print "Model size: ",size

In [ ]:
out.list_arguments()

In [ ]:
arg_shape, out_shape, aux_shape = out.infer_shape(X=(10, 3, 640,480), y=(10,3,270,110))
dict(zip(net.list_arguments(), arg_shape))
size = 0
for s in arg_shape:
    print s
    size += np.prod(s)
print "Total parameter size=%d MB, total temp size=%d MB" % (size / (1024*1024), 640*480*1000*8 / (1024*1024))

In [ ]:
net.list_arguments()

#### Note


 * Important Paper about Dilated Convolution: http://arxiv.org/pdf/1511.07122v1.pdf

 * Deep Probabilistic Inference Models http://gitxiv.com/posts/oRw692PEooNcwh9Qh/neural-variational-inference-for-text-processing



## Visualizing the reconstructed video ##

In [ ]:
import cv2

In [ ]:
!mkdir imageseries

In [ ]:
!rm imageseries/*.jpg

In [ ]:
frame.shape

In [ ]:
mask = tile_batch_mask((1,1,360, 640), 1.0, 4, 4)

In [ ]:
j = 0
c = 0

for i in permuted_indices:    
    #print np.sum(hsvimg-hsvimg2)
    fframe = inframes[i,:,:,:] # np.transpose(inframes[i,:,:,:], [1,2,0])
    fframe = fframe[np.newaxis,:,:,:]
    mfframe = np.concatenate((mask, fframe*mask), axis=1)
    print mfframe.shape
    mfframe = mfframe[0:1,:,100:164,100:164]
    clipfframe = fframe[0:1,:,100:164,100:164]
    bgi =  mx.io.NDArrayIter({'data' : mfframe}, batch_size=1)
    
    pframe = model.predict(bgi, num_batch=1)[0]
    print "Mean, Std"
    print np.mean(pframe[0,:,:])
    print np.mean(pframe[1,:,:])
    print np.mean(pframe[2,:,:])
    
    
    print np.std(pframe[0,:,:])
    print np.std(pframe[1,:,:])
    print np.std(pframe[2,:,:])
    
    pframe = np.clip(pframe, 0.0, 1.0, pframe)
    print pframe.shape
    print pframe.dtype
    
    px = pframe.shape[1]
    py = pframe.shape[2]
    offx = (clipfframe.shape[2]-px)/2
    offy = (clipfframe.shape[3]-py)/2
    print offx
    print offy
    
    mae = np.mean(np.abs(clipfframe[0,:,offx:px+offx,offy:py+offy]-pframe))
    print "MAE = %f" % (mae)
    
    pframe = (pframe * 255.0).astype(np.uint8)
    piframe = np.transpose(pframe, [1,2,0])
    bgrimg = cv2.cvtColor(piframe, cv2.COLOR_HSV2BGR)

    j+=1
    c+=1
    cv2.imwrite("imageseries/reconstruction1_%05d.jpg" % (c), bgrimg)
    if (c>400):
        break
    if (j % 10==0):
        print "frame %d" % (c)
    
    

In [ ]:
pframe

In [ ]:
from IPython.display import display, Image
display(Image('imageseries/reconstruction1_00002.jpg'))

In [ ]:
!rm imageseries/*.png

In [ ]:
!rm videos/Chimpanzees1_reconstructed3.mp4
!ffmpeg -i imageseries/reconstruction1_%05d.jpg -c:v libx264 -b:v 400k -maxrate:v 500k -bufsize 1000k -r 10 -preset medium videos/Chimpanzees1_reconstructed3.mp4


In [ ]:
from IPython.display import display, Image

In [ ]:
!ls -hs imageseries/recons*.jpg

In [ ]:
display(Image('imageseries/reconstruction1_00052.jpg'))

In [ ]:
link_video('videos/Chimpanzees1_reconstructed3.mp4')

In [ ]:
from IPython.display import FileLink
display(FileLink('videos/Chimpanzees1_reconstructed3.mp4'))

In [ ]:
link_video('videos/Chimpanzees1_reconstructed3.mp4')
#link_video('videos/Chimpanzees1_reconstructed2.mp4')

In [ ]:
link_video('videos/Chimpanzees1.mp4')

In [ ]:
display(FileLink('videos/Chimpanzees1_reconstructed2.mp4'))

In [ ]:
class MultiplyMaskOp(mx.operator.CustomOp):
    
    def forward(self, is_train, req, in_data, out_data, aux):
        img_channels = in_data[1].asnumpy()
        mask = in_data[0].asnumpy()
        
        if req[0]=='null':
            return
        
        shp = list(img_channels.shape)
        shp[1] += 1
        
        out = np.zeros(shape=tuple(shp))
        out[:,0:1,:,:] = mask
        out[:,1:,:,:] = img_channels*mask # Automatic broadcasting to the rescue
        
        if req[0]=='write':
            out_data[0][:] = out
        elif req[0]=='add':
            out_data[0][:] += out
                
    def backward(self, req, out_grad, in_data, out_data, in_grad, aux):
        if req[0]=='null':
            return
        
        mask = in_data[0]
        img_channels = in_data[1]
        
        img_grad = out_data[:,1:]
        mask_grad = out_data[:,0:1]
        
        # Gradient of mask is copied through
        
        if req[0]=='write':
            in_grad[0][:] = mask_grad
        elif req[0]=='add':
            in_grad[0][:] += mask_grad
           
        img_grad = np.ones(img_channels.shape)
        for i in range(3):
            # Product rule applies for image channels
            chan_grad = mx.nd.add(mx.nd.multiply(img_channels[:,i:i+1], mask_grad), 
                                  mx.nd.multiply(img_grad[:,i:i+1], mask) )
            img_grad[:,i,:,:] = chan.asnumpy()
            
        if req[1]=='write':
            in_grad[1][:] = img_grad    
        elif req[1]=='add':
            in_grad[1][:] += img_grad
            
@mx.operator.register("maskmult")
class MultiplyMaskOpProp(mx.operator.CustomOpProp):
    
    def __init__(self):
        super(MultiplyMaskOpProp, self).__init__()
        
    def list_arguments(self):
        return ['mask', 'data']

    def list_outputs(self):
        return ['masked_data']

    def infer_shape(self, in_shape):
        mask_shape = list(in_shape[0])
        data_shape = list(in_shape[1])
        data_shape[1]+=1
        return in_shape, [data_shape], []
    
    def create_operator(self, ctx, shapes, dtypes):
        return MultiplyMaskOp()
    

In [ ]:
(b*a).grad(2)